In [1]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor

In [2]:
POSTGRESQL_HOST = '10.129.0.25'

In [3]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


# Manual option

In [4]:
conn = psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
)
cur = conn.cursor()

In [5]:
query = 'SELECT * FROM seats LIMIT 10'

In [6]:
cur.execute(query)
records = cur.fetchall()
cur.close()
conn.close()

In [7]:
records

[('319', '2A', 'Business'),
 ('319', '2C', 'Business'),
 ('319', '2D', 'Business'),
 ('319', '2F', 'Business'),
 ('319', '3A', 'Business'),
 ('319', '3C', 'Business'),
 ('319', '3D', 'Business'),
 ('319', '3F', 'Business'),
 ('319', '4A', 'Business'),
 ('319', '4C', 'Business')]

# `with` context manager

In [8]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM seats LIMIT 5')
        records = cur.fetchall()

In [9]:
records

[('319', '2A', 'Business'),
 ('319', '2C', 'Business'),
 ('319', '2D', 'Business'),
 ('319', '2F', 'Business'),
 ('319', '3A', 'Business')]

# Find out who flew and where

## Who

In [10]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT passenger_id, passenger_name FROM Tickets WHERE ticket_no = '0005432312164'")
        passenger = cur.fetchall()



In [11]:
passenger

[('0423 646487', 'ALEKSEY GERASIMOV')]

## Where

In [12]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT flight_id FROM Ticket_flights WHERE ticket_no = '0005432312164'")
        flight = cur.fetchall()

In [13]:
flight

[(19976,), (136916,), (202076,), (202563,)]

In [14]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT departure_airport, arrival_airport FROM Flights WHERE flight_id = '19976' OR flight_id = '136916' OR flight_id = '202076' OR flight_id = '202563' ORDER BY scheduled_arrival ASC")
        airport = cur.fetchall()

In [15]:
airport

[('DME', 'EGO'), ('EGO', 'AAQ'), ('AAQ', 'EGO'), ('EGO', 'DME')]

# Conclusions
* The second connection option seems more convinient for me
* Not sure why there 4 `flight_id`s associated with the `ticket_no`
* The `ORDER BY` keyword with `ASC` really helped me understand the pattern (this is a ticket with many **connecting** flights)